## Agent
The core idea of agents is to use a language model to choose a sequence of actions to take. In chains, a sequence of actions is hardcoded (in code). In agents, a language model is used as a reasoning engine to determine which actions to take and in which order.

- agents 
    - are a combination of llms and code
    - llms reason about what step to take and call for actions

- agent loop
    - choose a tool to use
    - observe the output of the tool
    - repeat until a stopping conditon is met

- stopping conditions can be
    - llm determined
    - hardcoded rules


https://zhuanlan.zhihu.com/p/664281311

In [1]:
%env OPENAI_API_KEY=sk-7nEgUqufIHJhyUgfO2DzfV22uoS9AKTQ1lPQ96WoLDdTKg1b
%env OPENAI_URL=https://api.chatanywhere.tech/v1
import os
key=os.getenv('OPENAI_API_KEY')
url=os.getenv('OPENAI_URL')

env: OPENAI_API_KEY=sk-7nEgUqufIHJhyUgfO2DzfV22uoS9AKTQ1lPQ96WoLDdTKg1b
env: OPENAI_URL=https://api.chatanywhere.tech/v1


In [5]:
from langchain.globals import set_debug,set_verbose
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents import tool
from langchain.chat_models import ChatOpenAI
from langchain.agents import AgentExecutor,create_openai_tools_agent
set_debug(True)

set_verbose(True)

In [ ]:
import requests
from pydantic.v1 import BaseModel, Field
import datetime

# Define the input schema
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""
    
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
    # Parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_utc_time = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']
    
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]
    
    return f'The current temperature is {current_temperature}°C'

In [32]:
functions=[format_tool_to_openai_function(get_current_temperature)]

In [35]:
model_name='gpt-3.5-turbo'
model = ChatOpenAI(base_url=url,api_key=key,model=model_name).bind(functions=functions)
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser



[llm/start] [llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: what is the weather in sf right now"
  ]
}
[llm/end] [llm:ChatOpenAI] [1.10s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "",
        "generation_info": {
          "finish_reason": "function_call",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "",
            "additional_kwargs": {
              "function_call": {
                "arguments": "{\"latitude\":37.7749,\"longitude\":-122.4194}",
                "name": "get_current_temperature"
              }
            },
            "response_metadata": {
              "token_usage": {
                "completion_tokens": 25,
                "prompt

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":37.7749,"longitude":-122.4194}', 'name': 'get_current_temperature'}}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 95, 'total_tokens': 120}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-cc2ff5b9-ce19-4e3a-a07d-212b3519bbde-0')

In [30]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
model_name='gpt-3.5-turbo'
prompt=ChatPromptTemplate.from_messages([
            ("system","You are very powerful assistant,your name is benben, but bad at knowing the weather."),
            ("human","{input}"),
            MessagesPlaceholder(variable_name='agent_scratchpad')])



model = ChatOpenAI(base_url=url,api_key=key,model=model_name)
agent=create_openai_tools_agent(llm=model,tools=[get_current_temperature],prompt=prompt)

executor=AgentExecutor(agent=agent,tools=[get_current_temperature],verbose=True)
executor.invoke({'input':'what is your name?'})





> Entering new AgentExecutor chain...
My name is Benben. How can I assist you today?

> Finished chain.


{'input': 'what is your name?',
 'output': 'My name is Benben. How can I assist you today?'}

In [31]:
executor.invoke({'input':'what is the weather in Chengdu'})



> Entering new AgentExecutor chain...

Invoking: `get_current_temperature` with `{'latitude': 30.5728, 'longitude': 104.0668}`


The current temperature is 22.1°CThe current temperature in Chengdu is 22.1°C.

> Finished chain.


{'input': 'what is the weather in Chengdu',
 'output': 'The current temperature in Chengdu is 22.1°C.'}